In [30]:
import http.client
import json
import os
import time
import argparse

In [75]:
def _map(task_id, num_reduces):
    """
    Performs the map task. Saves the intermediate file to disk with a filename mr-{task_id}-{bucket_id}.txt.

    Parameters:
        task_id (int): ID of the task to be performed.
        num_reduces (int): Number of total reduce tasks.        
    """
    input = f'temp/{task_id}.txt'
    with open(input, 'r') as f:
        # We eliminate question marks, commas, etc...
        text = f.read().replace(";", " ").replace('"', " ").replace(",", " ").replace(".", " ").replace("!", " ").replace("?", " ").replace('-', ' ').replace('_', ' ').replace('[', ' ').replace(']', ' ').replace('(', ' ').replace(')', ' ').replace(":"," ").replace("*", " ").split()
         
    # Bucket words by the first letter modulo M
    for word in text:

        #We do not distinguish uppercase or lowercase. We also remove quotation marks and spaces
        word = word.lower().strip("'").strip()
        
        if word != '': # We add this to solve issues with empty words after eliminating '
        # We set the buket ID using the unicode code for the first character. 
            bucket_id = ord(word[0]) % num_reduces 
        
        # We append the words to the intermediate file 
            intermediate = f'intermediate/mr-{task_id}-{bucket_id}.txt'
            with open(intermediate, 'a') as bucket:
            
                bucket.write(f"{word}\n") 

In [83]:
def _reduce(bucket_id, num_maps):

    """
    Performs the reduce task for the given bucket_id and all the maps.

    Parameters:
        bucket_id (int): ID of the bucket on which the function performs the reduce task.
        num_maps (int): Number of map tasks that have produced buckets.
    """

    counts = {} # Empty dictionary to store the count

    # We run over the map tasks to collect all bucket files
    for map_id in range(num_maps): 
        intermediate = f'intermediate/mr-{map_id}-{bucket_id}.txt'
        if not os.path.exists(intermediate):
            continue
        with open(intermediate, 'r') as f:
            for word in f:
                word = word.strip()
                counts[word] = counts.get(word,0) + 1 # We add one to the word count in the dictionary
    
    
    
    for word, count in counts.items():
       print(f"{word}{count}\n")

    return counts

In [80]:
_map(1, 2)

In [84]:
cc = _reduce(1,5)

In [85]:
cc

{'engine': 1,
 'combined': 2,
 'so': 45,
 'imagine': 1,
 'standing': 3,
 'on': 54,
 'own': 5,
 'giving': 2,
 'orders': 3,
 'and': 418,
 'executing': 1,
 'stop': 4,
 'sir': 4,
 'a': 308,
 'almost': 4,
 'out': 37,
 'up': 42,
 'slowly': 1,
 'sidewalk': 1,
 'ship': 1,
 'arms': 4,
 'straightened': 1,
 'stiffened': 1,
 'sides': 1,
 'set': 5,
 'stabboard': 3,
 'chow': 7,
 'ch': 2,
 'wow': 1,
 'mean': 3,
 'stately': 1,
 'circles': 2,
 'it': 157,
 'was': 172,
 'wheel': 1,
 'go': 6,
 'come': 7,
 'ahead': 2,
 'your': 5,
 'outside': 4,
 'over': 24,
 'slow': 2,
 'ow': 2,
 'get': 7,
 'with': 96,
 'spring': 2,
 "what're": 1,
 'you': 62,
 'about': 18,
 'stump': 2,
 'of': 228,
 'stand': 1,
 'stage': 1,
 'engines': 1,
 "sh't": 2,
 "s'h't": 1,
 'gauge': 1,
 'cocks': 1,
 'went': 18,
 'whitewashing': 3,
 'attention': 5,
 'steamboat': 1,
 'stared': 1,
 'moment': 8,
 'said': 21,
 'yi': 1,
 "you're": 3,
 "ain't": 6,
 'answer': 3,
 'surveyed': 3,
 'eye': 5,
 'an': 19,
 'artist': 2,
 'gave': 8,
 'another': 6,
 

In [69]:
print(list(cc.keys())[10])

included



In [64]:
def _reduce(bucket_id, num_maps):

    """
    Performs the reduce task for the given bucket_id and all the maps.

    Parameters:
        bucket_id (int): ID of the bucket on which the function performs the reduce task.
        num_maps (int): Number of map tasks that have produced buckets.
    """

    counts = {} # Empty dictionary to store the count

    # We run over the map tasks to collect all bucket files
    for map_id in range(num_maps): 
        intermediate = f'intermediate/mr-{map_id}-{bucket_id}.txt'
        if not os.path.exists(intermediate):
            continue
        with open(intermediate, 'r') as f:
            for word in f:
                counts[word] = counts.get(word, 0) + 1 # We add one to the word count in the dictionary
    
    # Write the output to a file
    output = f'out/out-{bucket_id}.txt'
    with open(output, 'w') as f:
        for word, count in counts.items():
            f.write(f"{word} {count}")